In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.tokenize import word_tokenize as wt
from nltk.corpus import stopwords
nltk.download('stopwords')

import xgboost as xgb
from xgboost import XGBClassifier


In [107]:
from sklearn.gaussian_process import GaussianProcessClassifier

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [3]:
from sklearn.ensemble import RandomForestClassifier

In [4]:
from sklearn.model_selection import GridSearchCV

In [ ]:
from datetime import datetime

In [5]:
data = pd.read_csv("train.csv")

In [9]:
def c_set(df: pd.DataFrame):
    if 'ID' in df.columns:
        global dfID
        dfID = df['ID']
        df.drop(columns = 'ID', inplace=True)
        
    for i in range(len(df)):
        if 'first_party' in df.columns and 'second_party' in df.columns:
            if df.loc[i, 'first_party'] == 'United States' or df.loc[i, 'second_party'] == 'United_States':
                df.loc[i, 'US'] = 1
            else:
                df.loc[i, 'US'] = 0

    if 'first_party' in df.columns:
        global dffp
        dffp = df['first_party']
        df.drop(columns = 'first_party', inplace=True)
    if 'second_party' in df.columns:
        global dfsp
        dfsp = df['second_party']
        df.drop(columns = 'second_party', inplace=True)
        
    
    if 'first_party_winner' in df.columns:
        global target
        target = df['first_party_winner']
        df.drop(columns = 'first_party_winner', inplace=True)
    else:
        pass
        
    
        

In [10]:
c_set(data)

In [13]:
def vtr(data):
    stw = set(stopwords.words('english'))
    
#     n = re.compile("^[a-zA-Z]")
    num = re.compile("\d")
    for i in range(len(data)):
        text = re.sub(num, ' ', data.loc[i, 'facts'].lower())
        words = wt(text)
        
        texts = []
        for j in words:
            if j not in stw:
                    texts.append(j)
            
            try:
                int(j)
                
            except:
                continue
                
            else:
                texts.append(j)
                
            finally:
                continue
                
        phrase = " ".join(texts)
        phrase = phrase.replace(',', '')
        data.loc[i, 'facts'] = phrase
        
    min_df = 5
    max_features = 1500
    
    if 'tf' not in globals():

        global tf
        tf = TfidfVectorizer(min_df = min_df, max_features = max_features)
        
    if 'vct' not in globals():
        global vct
        vct = tf.fit_transform(data['facts'])
    else:
        global vcttest
        vcttest = tf.transform(data['facts'])
    
    if 'feature_names' not in globals():
        global feature_names
        feature_names = tf.get_feature_names_out()
    
    if 'vctdf' not in globals():
        global vctdf 
        vctdf = pd.DataFrame(vct.toarray(), columns=feature_names)
        vctdf = pd.concat([vctdf, data['US']], axis=1)
    
    else:
        global vctdft
        vctdft = pd.DataFrame(vcttest.toarray(), columns=feature_names)
        vctdft = pd.concat([vctdft, data['US']], axis=1)
    

In [14]:
test = pd.read_csv("test.csv")
submis = pd.read_csv("sample_submission.csv")

In [17]:
c_set(test)
vtr(test)

In [20]:
subid = submis['ID']
submis.drop(columns = 'ID', inplace=True)

In [116]:
def train(data=None, target=None, submission_data = None, submission_target=None):
    
    
    
    xtr, xte, ytr, yte = train_test_split(data, target)
    
    
#     params_grid_rfc = {
#         'min_samples_leaf': [2, 3, 5, 7],
#         'n_estimators': [3, 5, 7, 9, 10, 11, 12, 15]
#     }
    
    
#     global rfc, grid_search_rfc
#     rfc = RandomForestClassifier()
#     grid_search_rfc = GridSearchCV(rfc, param_grid = params_grid_rfc)
#     grid_search_rfc.fit(xtr, ytr)
#     grid_search_rfc.score(xtr, ytr)

#     rfctr = grid_search_rfc.best_estimator_.predict(xtr)
#     rfcte = grid_search_rfc.best_estimator_.predict(xte)

#     best_params_rfc = 'min_samples_leaf':7, 'n_estimators':15
#     best_params_xgb = {'learning_rate' = 0.03, 'max_depth'= 3, 'min_child_weight'=5, 'scale_pos_weight'=2}
    
    rfc = RandomForestClassifier()
#     xgbmodel = XGBClassifier(params = {'learning_rate' : 0.03, 
#                          'max_depth' : 3, 
#                          'min_child_weight' : 5, 
#                          'scale_pos_weight' : 2})
    
#     print("gridsearch_rfc best_params_ :", grid_search_rfc.best_params_)

    rfc.fit(xtr, ytr)
    rfctr = rfc.predict(xtr)
    print('rfc, train')
    print(accuracy_score(rfctr, ytr))

    rfcte = rfc.predict(xte)
    print('rfc, test')
    print(accuracy_score(rfcte, yte))

    global rfcpred
#     rfcpred = grid_search_rfc.best_estimator_.predict(submission_data)
    rfcpred = pd.DataFrame(rfc.predict(submission_data))
        
#     global grid_search_xgb
#     params_grid_xgb = {'max_depth': [3, 5, 7, 9, 10],
#          'learning_rate' : [0.01, 0.03, 0.05, 0.1],
#          'scale_pos_weight': [2],
#          'min_child_weight': [2, 3, 5]}

#     grid_search_xgb = GridSearchCV(XGBClassifier(), param_grid = params_grid_xgb, scoring='accuracy', cv=5)
#     grid_search_xgb.fit(xtr, ytr)

#     xgbtr = grid_search_xgb.best_estimator_.predict(xtr)
#     xgbte = grid_search_xgb.best_estimator_.predict(xte)

#     print('gridsearch_xgb best_params :', grid_search_xgb.best_params_)

    xgbmodel = XGBClassifier(learning_rate = 0.03, max_depth= 3, min_child_weight= 5, scale_pos_weight= 2)
    xgbmodel.fit(xtr, ytr)
    
    xgbtr = xgbmodel.predict(xtr)
    print('xgb, train')
    print(accuracy_score(xgbtr, ytr))

    xgbte = xgbmodel.predict(xte)
    print('xgb, test')
    print(accuracy_score(xgbte, yte))
    
    

    
    global xgbpred
#     xgbpred = grid_search_xgb.best_estimator_.predict(submission_data)
    xgbpred = pd.DataFrame(xgbmodel.predict(submission_data))
    
    if 'ID' in submission_target.columns:
        global subid
        subid = submission_target['ID']
        submission_target = submission_target.drop(columns = ['ID'])
    
    
    gpcmodel = GaussianProcessClassifier(n_jobs = 150)
    gpcmodel.fit(xtr, ytr)
    
    gpctr = gpcmodel.predict(xtr)
    print('gpc, train')
    print(accuracy_score(gpctr, ytr))
    
    gpcte = gpcmodel.predict(xte)
    print('gpc, test')
    print(accuracy_score(gpcte, yte))
    
    global gpcpred
    gpcpred = pd.DataFrame(gpcmodel.predict(submission_data))
    

    global submission_xgb
    submission_xgb = pd.concat([subid, xgbpred], axis=1)
    
    submission_xgb.columns = ['ID', 'first_party_winner']
    
    submission_xgb = submission_xgb.set_index("ID")
    
    
    global submission_rfc
    submission_rfc = pd.concat([subid, rfcpred], axis=1)
    
    submission_rfc.columns = ["ID", 'first_party_winner']
    
    submission_rfc = submission_rfc.set_index('ID')
    
    global submission_gpc
    submission_gpc = pd.concat([subid, gpcpred], axis=1)
    submission_gpc.columns = ['ID', 'first_party_winner']
    submissino_gpc = submission_gpc.set_index("ID")

In [241]:
train(vctdf, target, vctdft, submis)

rfc, train
0.9994617868675996
rfc, test
0.635483870967742
xgb, train
0.6722282023681377
xgb, test
0.646774193548387
gpc, train
0.6851453175457481
gpc, test
0.6483870967741936


In [115]:
submission_gpc.to_csv("{}gpc.csv".format(datetime.today().strftime("%m%d")))

In [246]:
submission_rfc.to_csv("{}rfc.csv".format(datetime.today().strftime("%m%d")))

In [ ]:
submission_gpc.to_csv("{}gpc.csv".format(datetime.today().strptime("%m%d")))

In [100]:
submission_xgb.to_csv("{}xgb.csv".format(datetime.today().strftime("%m%d")))

In [272]:
def sim_classifier(data, target, subdata):
    from sklearn.metrics.pairwise import linear_kernel
    cosine_sim = linear_kernel(data, subdata)
    cosine_sim = pd.DataFrame(cosine_sim)
    

    global answer
    answer = []
    
    for i in range(len(cosine_sim.columns)):
        m = max(cosine_train.loc[i, :])
        for j in range(len(cosine_sim.index)):
            if cosine_sim.loc[j, i] == m:
                answer.append(target[j])
                break
                
    answer = pd.DataFrame(answer)
    
    answer = pd.concat([subid, answer], axis=1)
    
    answer = answer.set_index("ID")
    

In [273]:
sim_classifier(vctdf, target, vctdft)

In [235]:
answer.columns = ['first_party_winner']

In [240]:
answer.to_csv("0703.csv")